In [1]:
from sklearn.datasets import load_breast_cancer
import pandas as pd
import numpy as np
from predictables.util import to_pl_lf
from predictables.univariate import Univariate
import numpy as np


# load data
df = load_breast_cancer(as_frame=True).frame
df.columns = [c.replace(" ", "_") for c in df.columns]
df["cv"] = np.random.randint(1, 6, df.shape[0])
df = to_pl_lf(df)

df.head().collect()

mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,radius_error,texture_error,perimeter_error,area_error,smoothness_error,compactness_error,concavity_error,concave_points_error,symmetry_error,fractal_dimension_error,worst_radius,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,f64,i32,i32
17.99,10.38,122.8,1001.0,0.1184,0.2776,0.3001,0.1471,0.2419,0.07871,1.095,0.9053,8.589,153.4,0.006399,0.04904,0.05373,0.01587,0.03003,0.006193,25.38,17.33,184.6,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.1189,0,2
20.57,17.77,132.9,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,0.5435,0.7339,3.398,74.08,0.005225,0.01308,0.0186,0.0134,0.01389,0.003532,24.99,23.41,158.8,1956.0,0.1238,0.1866,0.2416,0.186,0.275,0.08902,0,5
19.69,21.25,130.0,1203.0,0.1096,0.1599,0.1974,0.1279,0.2069,0.05999,0.7456,0.7869,4.585,94.03,0.00615,0.04006,0.03832,0.02058,0.0225,0.004571,23.57,25.53,152.5,1709.0,0.1444,0.4245,0.4504,0.243,0.3613,0.08758,0,4
11.42,20.38,77.58,386.1,0.1425,0.2839,0.2414,0.1052,0.2597,0.09744,0.4956,1.156,3.445,27.23,0.00911,0.07458,0.05661,0.01867,0.05963,0.009208,14.91,26.5,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.173,0,4
20.29,14.34,135.1,1297.0,0.1003,0.1328,0.198,0.1043,0.1809,0.05883,0.7572,0.7813,5.438,94.44,0.01149,0.02461,0.05688,0.01885,0.01756,0.005115,22.54,16.67,152.2,1575.0,0.1374,0.205,0.4,0.1625,0.2364,0.07678,0,1


In [2]:
df.collect().shape

(569, 32)

In [3]:
from sklearn.decomposition import PCA

# Specify the number of components for the lower-dimensional subspace
n_components = 2

# Create an instance of PCA
pca = PCA(n_components=n_components)

# Fit the PCA model to the data and transform the data
df_projected = pca.fit_transform(
    df.collect().to_pandas().drop(columns=["cv", "target"])
)

# Convert the projected data into a new dataframe
df_pca = pd.DataFrame(
    df_projected, columns=[f"Component {i+1}" for i in range(n_components)]
)

df_pca["cv"] = df.collect().to_pandas()["cv"]
df_pca["target"] = df.collect().to_pandas()["target"]

df_pca.head()

,Component 1,Component 2,cv,target
0,1160.142574,-293.917544,2,0
1,1269.122443,15.630182,5,0
2,995.793889,39.156743,4,0
3,-407.180803,-67.380320,4,0
4,930.341180,189.340742,1,0


### fit each fold to a model

In [4]:
X, y = (
    df.collect().to_pandas().drop(columns="target"),
    df.select("target").collect().to_pandas()["target"],
)
X.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_texture,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,2
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,23.41,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,5
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,25.53,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,4
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,4
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,16.67,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,1


In [5]:
df = df.collect().to_pandas()
df.head()

,mean_radius,mean_texture,mean_perimeter,mean_area,mean_smoothness,mean_compactness,mean_concavity,mean_concave_points,mean_symmetry,mean_fractal_dimension,...,worst_perimeter,worst_area,worst_smoothness,worst_compactness,worst_concavity,worst_concave_points,worst_symmetry,worst_fractal_dimension,target,cv
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,0,2
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,158.80,1956.0,0.1238,0.1866,0.2416,0.1860,0.2750,0.08902,0,5
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,152.50,1709.0,0.1444,0.4245,0.4504,0.2430,0.3613,0.08758,0,4
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,0,4
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,0.1980,0.10430,0.1809,0.05883,...,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,0,1


In [6]:
# filename = f'./test_{np.random.randint(0, 123456789)}.pdf'
# margins = [0.25, 0.25, 0.5, 0.5]

# ua = Univariate(df, 'cv', 'mean_radius', 'target', time_series_validation=False)

# func = lambda: ua.plot_cdf(figsize=(6,6))

# Report(filename, margins).plot_no_caption(func(), 6, 6)

In [7]:
from tqdm import tqdm

filename = f"./test_{np.random.randint(0, 123456789)}.pdf"
margins = [0.25, 0.25, 0.5, 0.5]

rpt = (
    Univariate._rpt_title_page(filename=filename, margins=margins)
    .title("Univariate Analysis - Hit Ratio Model")
    .author("Andy Weaver")
)

for col in tqdm(df_pca.columns[:2]):
    if col not in ["target", "cv"]:
        ua = Univariate(df_pca, "cv", col, "target", time_series_validation=False)
        rpt = ua._add_to_report(rpt)


rpt.build()

  0%|          | 0/2 [00:00<?, ?it/s]

100%|██████████| 2/2 [00:14<00:00,  7.30s/it]


In [8]:
ua = Univariate(df_pca, "cv", "Component 1", "target", time_series_validation=False)
print(ua.get_results())

                    Fold-1    Fold-2    Fold-3    Fold-4    Fold-5 Agg. Mean  \
Fitted Coef.      -7.4e-03  -7.0e-03  -7.2e-03  -7.5e-03  -7.2e-03  -7.3e-03   
Fitted p-Value     8.9e-27   2.9e-27   7.3e-27   2.9e-25   5.4e-26   1.2e-32   
Fitted Std. Err.   6.9e-04   6.5e-04   6.7e-04   7.3e-04   6.8e-04   6.1e-04   
Conf. Int. Lower  -8.8e-03  -8.3e-03  -8.5e-03  -9.0e-03  -8.6e-03  -8.5e-03   
Conf. Int. Upper  -6.1e-03  -5.8e-03  -5.9e-03  -6.1e-03  -5.9e-03  -6.1e-03   
Train Accuracy       91.5%     90.7%     90.6%     89.8%     90.9%     90.7%   
Val Accuracy         87.0%     90.8%     91.2%     94.1%     90.0%     90.7%   
Train AUC            89.8%     88.5%     88.2%     87.6%     89.1%     88.6%   
Val AUC              84.1%     89.5%     90.2%     92.6%     86.8%     88.6%   
Train F1             93.5%     92.8%     92.9%     92.2%     92.8%     92.9%   
Test F1              90.1%     93.0%     92.8%     95.4%     92.9%     92.9%   
Train Precision      90.9%     88.9%    

In [9]:
ua.pareto_sort_vector

[[0.8864951641033771, 0.03247873816841429],
 [0.9068541300527241, 0.02538653454069585],
 [0.9286675639300135, 0.018498932200684198],
 [0.8937823834196891, 0.03541135647598336],
 [0.9663865546218487, 0.024246790160422724],
 [0.8001232702095267, 0.05505633344359451]]

In [13]:
harmonic_mean?

Signature: harmonic_mean(*args: float) -> float
Docstring:
Returns the harmonic mean of the given numbers.

Parameters
----------
*args : numeric
    Numbers to be averaged.

Returns
-------
float
    The harmonic mean of the given numbers.
File:      c:\users\aweaver\predictables\predictables\util\src\_harmonic_mean.py
Type:      function

In [12]:
from predictables.util import harmonic_mean
pd.DataFrame(ua.pareto_sort_vector).apply(lambda x: harmonic_mean(x), axis=1)

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

In [16]:
import polars as pl
dir(pl.Series)

['__abs__',
 '__add__',
 '__and__',
 '__annotations__',
 '__array__',
 '__array_ufunc__',
 '__bool__',
 '__class__',
 '__column_consortium_standard__',
 '__contains__',
 '__copy__',
 '__deepcopy__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__floordiv__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getitem__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__invert__',
 '__iter__',
 '__le__',
 '__len__',
 '__lt__',
 '__matmul__',
 '__mod__',
 '__module__',
 '__mul__',
 '__ne__',
 '__neg__',
 '__new__',
 '__or__',
 '__pos__',
 '__pow__',
 '__radd__',
 '__rand__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__rfloordiv__',
 '__rmatmul__',
 '__rmod__',
 '__rmul__',
 '__ror__',
 '__rpow__',
 '__rsub__',
 '__rtruediv__',
 '__rxor__',
 '__setattr__',
 '__setitem__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__sub__',
 '__subclasshook__',
 '__truediv__',
 '__weakref__',
 '__xor__',
 '_accessors',
 '_arithmetic',
 '_comp',
 